## OpenAI

In [1]:
import os
import base64
import json
import csv
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI
from kenya.DietAI24_OpenAI import DietAI24

In [ ]:
model_name = "gpt-4.1-mini"
path_images = "../data/kenya/kenya_parsed_food_images"
path_results = f"../output/kenya_dietai24_{model_name}.csv"
path_food_list = "../data/kenya/kenya_food_list.txt"

In [3]:
os.makedirs(os.path.dirname(path_results), exist_ok=True)
ls_images = os.listdir(path_images)
ls_images = [os.path.join(path_images, x) for x in ls_images if x.endswith(".png")]
fields = ["image", "food_name", "food_confidence", "food_reasoning", "weight_grams", "weight_confidence", "weight_reasoning"]

In [ ]:
with open(path_results, "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()

for image_path in tqdm(ls_images, desc="Processing images"):
    row = {"image": os.path.basename(image_path)}

    recognizer = DietAI24(model_name=model_name, food_list_path=path_food_list)

    try:
        food_response = recognizer.recognize_food(image_path)
        food_data = json.loads(food_response)
        row["food_name"] = food_data.get("food_name")
        row["food_confidence"] = food_data.get("confidence")
        row["food_reasoning"] = food_data.get("reasoning")
    except Exception as e:
        row["food_name"] = None
        row["food_confidence"] = None
        row["food_reasoning"] = f"Errors: {str(e)}"

    if row["food_name"] and str(row["food_name"]).lower() != "null":
        try:
            weight_response = recognizer.estimate_weight(
                assumed_food=row["food_name"], image_path=image_path
            )
            weight_data = json.loads(weight_response)
            row["weight_grams"] = weight_data.get("weight_grams")
            row["weight_confidence"] = weight_data.get("confidence")
            row["weight_reasoning"] = weight_data.get("reasoning")
        except Exception as e:
            row["weight_grams"] = None
            row["weight_confidence"] = None
            row["weight_reasoning"] = f"Errors: {str(e)}"
    else:
        row["weight_grams"] = None
        row["weight_confidence"] = None
        row["weight_reasoning"] = "Food not recognized or null."

    recognizer.reset_conversation()  # Safe, optional

    # Write this row to CSV immediately (append mode)
    with open(path_results, "a", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fields)
        writer.writerow(row)

print(f"Results saved to {path_results}")

Processing images:   0%|          | 0/464 [00:00<?, ?it/s]

Processing images:  21%|██▏       | 99/464 [12:02<44:22,  7.29s/it]  

{
  "food_name": "Fusilli",
  "reasoning": "The food on the plate consists of spiral-shaped pasta, which matches the appearance of fusilli. Fusilli is listed among the common Kenyan foods provided.",
  "confidence": 0.95
}
Results saved to ../output/kenya_dietai24_gpt-4.1.csv
